<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [1]:
import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


Read data created in the previous chapter.

In [4]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [5]:
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [6]:
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

Our input function for predictions is the same except we don't provide a label

In [7]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

### Create feature columns for estimator

In [8]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [9]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_is_chief': True, '_save_summary_steps': 100, '_master': '', '_save_checkpoints_steps': None, '_model_dir': 'taxi_trained', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f058368d400>, '_num_worker_replicas': 1, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_train_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_log_step_count_steps': 100, '_task_type': 'worker', '_keep_checkpoint_max': 5, '_task_id': 0, '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 216

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [10]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-10-16:24:54
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-10-16:24:54
INFO:tensorflow:Saving dict for global step 608: average_loss = 109.73099, global_step = 608, loss = 13050.149
RMSE on dataset = 10.475255966186523


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [11]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([10.676832], dtype=float32)}
{'predictions': array([10.674335], dtype=float32)}
{'predictions': array([10.6751795], dtype=float32)}
{'predictions': array([10.67278], dtype=float32)}
{'predictions': array([10.676576], dtype=float32)}
{'predictions': array([10.67636], dtype=float32)}
{'predictions': array([10.674968], dtype=float32)}
{'predictions': array([10.674999], dtype=float32)}
{'predictions': array([10.676727], dtype=float32)}
{'predictions': array([10.674623], dtype=float32)}
{'predictions': array([10.676833], dtype=float32)}
{'predictions': array([10.676995], dtype=float32)}
{'predictions': array([10.670404], dtype=float32)}
{'predictions': array([10.674248], dtype=float32)}
{'predictions': array([1

{'predictions': array([10.780386], dtype=float32)}
{'predictions': array([10.66984], dtype=float32)}
{'predictions': array([10.667277], dtype=float32)}
{'predictions': array([10.672667], dtype=float32)}
{'predictions': array([10.674281], dtype=float32)}
{'predictions': array([10.657251], dtype=float32)}
{'predictions': array([10.674545], dtype=float32)}
{'predictions': array([10.660483], dtype=float32)}
{'predictions': array([10.727327], dtype=float32)}
{'predictions': array([10.676691], dtype=float32)}
{'predictions': array([10.676186], dtype=float32)}
{'predictions': array([10.728803], dtype=float32)}
{'predictions': array([10.723026], dtype=float32)}
{'predictions': array([10.729751], dtype=float32)}
{'predictions': array([10.673395], dtype=float32)}
{'predictions': array([10.670549], dtype=float32)}
{'predictions': array([10.781134], dtype=float32)}
{'predictions': array([10.673115], dtype=float32)}
{'predictions': array([10.676498], dtype=float32)}
{'predictions': array([10.675717

{'predictions': array([10.67734], dtype=float32)}
{'predictions': array([10.672395], dtype=float32)}
{'predictions': array([10.727286], dtype=float32)}
{'predictions': array([10.674304], dtype=float32)}
{'predictions': array([10.887005], dtype=float32)}
{'predictions': array([10.718858], dtype=float32)}
{'predictions': array([10.675241], dtype=float32)}
{'predictions': array([10.676542], dtype=float32)}
{'predictions': array([10.713818], dtype=float32)}
{'predictions': array([10.675009], dtype=float32)}
{'predictions': array([10.728716], dtype=float32)}
{'predictions': array([10.67601], dtype=float32)}
{'predictions': array([10.727617], dtype=float32)}
{'predictions': array([10.780736], dtype=float32)}
{'predictions': array([10.675716], dtype=float32)}
{'predictions': array([10.6705065], dtype=float32)}
{'predictions': array([10.673665], dtype=float32)}
{'predictions': array([10.779986], dtype=float32)}
{'predictions': array([10.779377], dtype=float32)}
{'predictions': array([10.677507

{'predictions': array([10.67772], dtype=float32)}
{'predictions': array([10.675341], dtype=float32)}
{'predictions': array([10.675876], dtype=float32)}
{'predictions': array([10.675879], dtype=float32)}
{'predictions': array([10.674946], dtype=float32)}
{'predictions': array([10.675711], dtype=float32)}
{'predictions': array([10.674847], dtype=float32)}
{'predictions': array([10.674982], dtype=float32)}
{'predictions': array([10.675173], dtype=float32)}
{'predictions': array([10.675968], dtype=float32)}
{'predictions': array([10.727814], dtype=float32)}
{'predictions': array([10.674239], dtype=float32)}
{'predictions': array([10.674649], dtype=float32)}
{'predictions': array([10.673784], dtype=float32)}
{'predictions': array([10.673866], dtype=float32)}
{'predictions': array([10.728855], dtype=float32)}
{'predictions': array([10.675285], dtype=float32)}
{'predictions': array([10.67513], dtype=float32)}
{'predictions': array([10.676122], dtype=float32)}
{'predictions': array([10.674173]

{'predictions': array([10.782103], dtype=float32)}
{'predictions': array([10.779784], dtype=float32)}
{'predictions': array([10.671338], dtype=float32)}
{'predictions': array([10.887751], dtype=float32)}
{'predictions': array([10.72831], dtype=float32)}
{'predictions': array([10.727335], dtype=float32)}
{'predictions': array([10.67555], dtype=float32)}
{'predictions': array([10.781381], dtype=float32)}
{'predictions': array([10.884239], dtype=float32)}
{'predictions': array([10.675655], dtype=float32)}
{'predictions': array([10.674251], dtype=float32)}
{'predictions': array([10.670118], dtype=float32)}
{'predictions': array([10.675931], dtype=float32)}
{'predictions': array([10.729468], dtype=float32)}
{'predictions': array([10.674927], dtype=float32)}
{'predictions': array([10.67489], dtype=float32)}
{'predictions': array([10.682127], dtype=float32)}
{'predictions': array([10.886105], dtype=float32)}
{'predictions': array([10.727676], dtype=float32)}
{'predictions': array([10.938378],

{'predictions': array([10.674795], dtype=float32)}
{'predictions': array([10.782178], dtype=float32)}
{'predictions': array([10.7806635], dtype=float32)}
{'predictions': array([10.672529], dtype=float32)}
{'predictions': array([10.676619], dtype=float32)}
{'predictions': array([10.674264], dtype=float32)}
{'predictions': array([10.726105], dtype=float32)}
{'predictions': array([10.6761], dtype=float32)}
{'predictions': array([10.6742], dtype=float32)}
{'predictions': array([10.731091], dtype=float32)}
{'predictions': array([10.727735], dtype=float32)}
{'predictions': array([10.887245], dtype=float32)}
{'predictions': array([10.675081], dtype=float32)}
{'predictions': array([10.674889], dtype=float32)}
{'predictions': array([10.676163], dtype=float32)}
{'predictions': array([10.727281], dtype=float32)}
{'predictions': array([10.674086], dtype=float32)}
{'predictions': array([10.6742735], dtype=float32)}
{'predictions': array([10.728858], dtype=float32)}
{'predictions': array([10.675125]

{'predictions': array([10.830831], dtype=float32)}
{'predictions': array([10.677471], dtype=float32)}
{'predictions': array([10.675351], dtype=float32)}
{'predictions': array([10.674216], dtype=float32)}
{'predictions': array([10.674045], dtype=float32)}
{'predictions': array([10.939559], dtype=float32)}
{'predictions': array([10.656252], dtype=float32)}
{'predictions': array([10.676712], dtype=float32)}
{'predictions': array([10.886937], dtype=float32)}
{'predictions': array([10.675218], dtype=float32)}
{'predictions': array([10.675569], dtype=float32)}
{'predictions': array([10.660912], dtype=float32)}
{'predictions': array([10.67523], dtype=float32)}
{'predictions': array([10.675024], dtype=float32)}
{'predictions': array([10.728277], dtype=float32)}
{'predictions': array([10.6748295], dtype=float32)}
{'predictions': array([10.674527], dtype=float32)}
{'predictions': array([10.675366], dtype=float32)}
{'predictions': array([10.676324], dtype=float32)}
{'predictions': array([10.72665

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [12]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 100));
print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_is_chief': True, '_save_summary_steps': 100, '_master': '', '_save_checkpoints_steps': None, '_model_dir': 'taxi_trained', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f055e2e6e10>, '_num_worker_replicas': 1, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_train_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_log_step_count_steps': 100, '_task_type': 'worker', '_keep_checkpoint_max': 5, '_task_id': 0, '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 415

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [13]:
import datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key,
  DAYOFWEEK(pickup_datetime)*1.0 AS dayofweek,
  HOUR(pickup_datetime)*1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 == {1}".format(base_query, phase)
  else:
    query = "{0} AND ABS(HASH(pickup_datetime)) % {1} == {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).to_dataframe()

In [14]:
print_rmse(model, df)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-10-16:28:38
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-6071
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-10-16:28:40
INFO:tensorflow:Saving dict for global step 6071: average_loss = 111.67871, global_step = 6071, loss = 14194.65
RMSE on dataset = 10.567814826965332


RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import shutil
import math

In [ ]:
# create datasets: train, validation, test
df_train = pd.DataFrame(1.5 * np.random.random_sample((10000,2)) + 0.5, 
            columns = ['radius','height'])

df_valid = pd.DataFrame(1.5 * np.random.random_sample((1000,2)) + 0.5, 
            columns = ['radius','height'])

df_test = pd.DataFrame(1.5 * np.random.random_sample((1000,2)) + 0.5, 
            columns = ['radius','height'])

In [ ]:
# Calculate and append volume to datasets

# training data
labels = []
for index, row in df_train.iterrows():
  volume = math.pi * row['radius']**2 * row['height']
  labels.append(volume)
df_train.insert(0, 'volume', labels, True)

# validation data
labels.clear()
for index, row in df_valid.iterrows():
  volume = math.pi * row['radius']**2 * row['height']
  labels.append(volume)
df_valid.insert(0, 'volume', labels, True)

# test data
labels.clear()
for index, row in df_test.iterrows():
  volume = math.pi * row['radius']**2 * row['height']
  labels.append(volume)
df_test.insert(0, 'volume', labels, True)

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License